In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import copy
import os
import yaml
from datetime import timedelta

import sys
sys.path.append('../../')

from data.processing.processing import get_data_from_tracker

from utils.generic.config import read_config
from utils.fitting.smooth_jump import smooth_big_jump

from viz.data import plot_data
from utils.generic.enums.columns import *
from viz.utils import setup_plt, axis_formatter

In [ ]:
config_filename = 'default.yaml'
config = read_config(config_filename)

In [ ]:
df = get_data_from_tracker(state='Maharashtra', district='Mumbai', use_dataframe='data_all')['data_frame']
df = df[df['date'] <= '2020-07-31']

In [ ]:
# Perform smoothing
print('smoothing params', config['fitting']['data']['smooth_jump_params'])
df_smooth, _ = smooth_big_jump(df, config['fitting']['data']['smooth_jump_params'])

In [ ]:
fig = plot_data(df)

In [ ]:
import matplotlib as mpl
## for Palatino and other serif fonts use:
plt.rcParams.update({
    'text.usetex': True,
    'font.size': 20,
    'font.family': 'Palatino',
 })

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
for comp in ['active', 'total', 'recovered', 'deceased']:
    compartment = Columns.from_name(comp)
    ax.plot(df[compartments['date'][0].name].to_numpy(), df[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='Simulated Data, Unspiked ({})'.format(compartment.label))
    ax.plot(df_smooth[compartments['date'][0].name].to_numpy(), df_smooth[compartment.name].to_numpy(),
            '-', color=compartment.color, label='Smoothed Data ({})'.format(compartment.label))
legend_elements = [
    Line2D([0], [0], ls='-', marker='o', ms=5, color='black', label='Original Data'),
    Line2D([0], [0], ls='-', color='black', label='Smoothed Data')
]
first_legend = ax.legend(handles=legend_elements, loc='upper left')
ax.add_artist(first_legend)
legend_elements = [
    Line2D([0], [0], ls='-', color='C0', label=f'Confirmed Cases'),
    Line2D([0], [0], ls='-', color='orange', label=f'Active Cases'),
    Line2D([0], [0], ls='-', color='green', label=f'Recovered'),
    Line2D([0], [0], ls='-', color='red', label=f'Deceased'),
]
ax.legend(handles=legend_elements, loc=[0.01, 0.7])
axis_formatter(ax)
ax.set_title('Smoothing Algorithm Illustration for Mumbai, India')
plt.tight_layout()
fig.savefig(f'../../../paper/plots/smoothing-mumbai.pdf', format='pdf', bbox_inches='tight', pad_inches=0)